In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

def load_data(filename):
    """Loads the preprocessed dataset."""
    df = pd.read_csv(filename)
    df = df.select_dtypes(include=[np.number]).dropna()  # Ensure only numeric columns
    X = df.drop(columns=["Price"], errors="ignore")  # Features
    y = df["Price"] if "Price" in df.columns else df.iloc[:, -1]  # Target
    
    return train_test_split(X, y, test_size=0.2, random_state=42)

def train_mlp(X_train, X_test, y_train, y_test, hidden_layers=(128, 64), max_iter=500):
    """Builds, trains, and evaluates an MLP model using Scikit-Learn."""

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Define MLP model
    mlp = MLPRegressor(hidden_layer_sizes=hidden_layers, activation="relu", solver="adam",
                        max_iter=max_iter, random_state=42)

    # Train model
    mlp.fit(X_train, y_train)

    # Make predictions
    y_pred = mlp.predict(X_test)

    # Evaluate model
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Test MAE: {mae:.2f}")

    return mlp

# Load the csv file in
filename1="vietnam_housing_dataset.csv"
X_train, X_test, y_train, y_test = load_data(filename1)

# Train the MLP model using Scikit-Learn
mlp_model1 = train_mlp(X_train, X_test, y_train, y_test, hidden_layers=(128, 64), max_iter=500)


Test MAE: 1.45


In [6]:
# Load the preprocessed dataset from DoAn.ipynb
filename2="vietnam_housing_dataset_cleaned_data.csv"
X_train, X_test, y_train, y_test = load_data(filename2)

# Train the MLP model using Scikit-Learn
mlp_model2 = train_mlp(X_train, X_test, y_train, y_test, hidden_layers=(128, 64), max_iter=500)


Test MAE: 1.42
